In [4]:
import os

In [2]:
from transformers import Wav2Vec2Processor, Data2VecAudioModel
import torch
from torch import nn
from datasets import load_dataset
import numpy
import pandas
import seaborn
from sklearn import cluster
from matplotlib import pyplot
import sklearn
import pandas
from scipy.stats import kendalltau
import librosa
import hdbscan
from sklearn.cluster import AgglomerativeClustering

In [5]:
d = './data_sub_mp3/'
listed = [os.path.abspath('{0}{1}'.format(d, x)) for x in os.listdir(d)]
'desktop.ini' in listed

False

In [4]:
[x for x in listed if 'desktop.ini' in x]

[]

In [6]:
x_s, sr_s, xx_s, xxdb_s, spectral_centroids_s, spectral_rolloff_s = [], [], [], [], [], []
spectral_bandwidth_2_s, spectral_bandwidth_3_s, spectral_bandwidth_4_s = [], [], []
zero_crossings_s, mfccs_s, mfccs10_s, chromagram_s = [], [], [], []
with torch.no_grad():
    for j in range(len(listed)):
        print('{0} / {1}'.format(j, len(listed)))
        audio_data = listed[j]
        x, sr = librosa.load(audio_data, sr=None)

        xx = librosa.stft(x)
        xxdb = librosa.amplitude_to_db(abs(xx))

        spectral_centroids = librosa.feature.spectral_centroid(y=x, sr=sr)[0]

        spectral_rolloff = librosa.feature.spectral_rolloff(y=x+0.01, sr=sr)[0]

        spectral_bandwidth_2 = librosa.feature.spectral_bandwidth(y=x+0.01, sr=sr)[0]
        spectral_bandwidth_3 = librosa.feature.spectral_bandwidth(y=x+0.01, sr=sr, p=3)[0]
        spectral_bandwidth_4 = librosa.feature.spectral_bandwidth(y=x+0.01, sr=sr, p=4)[0]

        zero_crossings = librosa.zero_crossings(y=x, pad=False)

        mfccs = librosa.feature.mfcc(y=x, sr=sr)
        mfccs10 = librosa.feature.mfcc(y=x, sr=sr, n_mfcc=5)

        chromagram = librosa.feature.chroma_stft(y=x, sr=sr)

        x_s.append(x)
        sr_s.append(sr)
        xx_s.append(xx)
        xxdb_s.append(xxdb)
        spectral_centroids_s.append(spectral_centroids)
        spectral_rolloff_s.append(spectral_rolloff)
        spectral_bandwidth_2_s.append(spectral_bandwidth_2)
        spectral_bandwidth_3_s.append(spectral_bandwidth_3)
        spectral_bandwidth_4_s.append(spectral_bandwidth_4)
        zero_crossings_s.append(zero_crossings)
        mfccs_s.append(mfccs)
        mfccs10_s.append(mfccs10)
        chromagram_s.append(chromagram)


0 / 4
1 / 4
2 / 4
3 / 4


In [7]:
def multivariate_calculator(var1, var2, measure, multi_agg_func):
    if len(var1.shape) != len(var2.shape):
        raise Exception()
    if len(var1.shape) == 1:
        var1 = var1.reshape(-1, 1)
        var2 = var2.reshape(-1, 1)
    measured = []
    for j in range(var1.shape[1]):
        m = measure(var1[:, j], var2[:, j])
        measured.append(m)
    measured_agg = multi_agg_func(measured)
    return measured_agg


In [8]:
def dtw_kt(x1, x2):
    gg = librosa.sequence.dtw(X=x1, Y=x2)
    y1, y2 = [], []
    for j in range(gg[1][:, 1].shape[0]):
        y1.append(x1[gg[1][:, 0][j]])
        y2.append(x2[gg[1][:, 1][j]])
    lx = pandas.DataFrame(data={'y1': y1, 'y2': y2})
    result = kendalltau(x=lx['y1'].values, y=lx['y2'].values)[0]
    return result

In [204]:
import time
# current_time = time.time()
# print('xxdb')
# xxdb_z = multivariate_calculator(var1=xxdb_s[0].T, var2=xxdb_s[1].T, measure=dtw_kt, multi_agg_func=numpy.mean)
# run_time = time.time() - current_time
# print('run time: {0} s'.format(int(run_time)))
current_time = time.time()
print('spectral_centroids')
spectral_centroids_z = multivariate_calculator(var1=spectral_centroids_s[0], var2=spectral_centroids_s[1], measure=dtw_kt, multi_agg_func=numpy.mean)
run_time = time.time() - current_time
print('run time: {0} s'.format(int(run_time)))
current_time = time.time()
print('spectral_rolloff')
spectral_rolloff_z = multivariate_calculator(var1=spectral_rolloff_s[0], var2=spectral_rolloff_s[1], measure=dtw_kt, multi_agg_func=numpy.mean)
run_time = time.time() - current_time
print('run time: {0} s'.format(int(run_time)))
current_time = time.time()
print('spectral_bandwidth_2')
spectral_bandwidth_2_z = multivariate_calculator(var1=spectral_bandwidth_2_s[0], var2=spectral_bandwidth_2_s[1], measure=dtw_kt, multi_agg_func=numpy.mean)
run_time = time.time() - current_time
print('run time: {0} s'.format(int(run_time)))
current_time = time.time()
print('spectral_bandwidth_3')
spectral_bandwidth_3_z = multivariate_calculator(var1=spectral_bandwidth_3_s[0], var2=spectral_bandwidth_3_s[1], measure=dtw_kt, multi_agg_func=numpy.mean)
run_time = time.time() - current_time
print('run time: {0} s'.format(int(run_time)))
current_time = time.time()
print('spectral_bandwidth_4')
spectral_bandwidth_4_z = multivariate_calculator(var1=spectral_bandwidth_4_s[0], var2=spectral_bandwidth_4_s[1], measure=dtw_kt, multi_agg_func=numpy.mean)
run_time = time.time() - current_time
print('run time: {0} s'.format(int(run_time)))
current_time = time.time()
print('zero_crossings')
zero_crossings_z = abs(abs(sum(zero_crossings_s[0]) / zero_crossings_s[0].shape[0]) - abs(sum(zero_crossings_s[1]) / zero_crossings_s[1].shape[0]))
run_time = time.time() - current_time
print('run time: {0} s'.format(int(run_time)))
current_time = time.time()
print('mfccs')
mfccs_z = multivariate_calculator(var1=mfccs_s[0].T, var2=mfccs_s[1].T, measure=dtw_kt, multi_agg_func=numpy.mean)
run_time = time.time() - current_time
print('run time: {0} s'.format(int(run_time)))
current_time = time.time()
print('mfccs10')
mfccs10_z = multivariate_calculator(var1=mfccs10_s[0].T, var2=mfccs10_s[1].T, measure=dtw_kt, multi_agg_func=numpy.mean)
run_time = time.time() - current_time
print('run time: {0} s'.format(int(run_time)))
current_time = time.time()
print('chromagram')
chromagram_z = multivariate_calculator(var1=chromagram_s[0].T, var2=chromagram_s[1].T, measure=dtw_kt, multi_agg_func=numpy.mean)
run_time = time.time() - current_time
print('run time: {0} s'.format(int(run_time)))

spectral_centroids
run time: 1 s
spectral_rolloff
run time: 1 s
spectral_bandwidth_2
run time: 1 s
spectral_bandwidth_3
run time: 4 s
spectral_bandwidth_4
run time: 2 s
zero_crossings
run time: 0 s
mfccs
run time: 32 s
mfccs10
run time: 7 s
chromagram
run time: 19 s


In [9]:
corr_spectral_centroids = numpy.full(shape=(len(x_s), len(x_s)), fill_value=numpy.nan)
for i in range(len(x_s)):
    print('{0} / {1}'.format(i, len(x_s)))
    for j in range(len(x_s)):
        spectral_centroids_z = multivariate_calculator(var1=spectral_centroids_s[i], var2=spectral_centroids_s[j], measure=dtw_kt, multi_agg_func=numpy.mean)
        corr_spectral_centroids[i, j] = spectral_centroids_z
corr_spectral_centroids

0 / 4
1 / 4



KeyboardInterrupt



In [ ]:
corr_spectral_rolloff = numpy.full(shape=(len(x_s), len(x_s)), fill_value=numpy.nan)
for i in range(len(x_s)):
    print('{0} / {1}'.format(i, len(x_s)))
    for j in range(len(x_s)):
        spectral_rolloff_z = multivariate_calculator(var1=spectral_rolloff_s[i], var2=spectral_rolloff_s[j], measure=dtw_kt, multi_agg_func=numpy.mean)
        corr_spectral_rolloff[i, j] = spectral_rolloff_z
corr_spectral_rolloff

In [ ]:
corr_spectral_bandwidth_2 = numpy.full(shape=(len(x_s), len(x_s)), fill_value=numpy.nan)
for i in range(len(x_s)):
    print('{0} / {1}'.format(i, len(x_s)))
    for j in range(len(x_s)):
        spectral_bandwidth_2_z = multivariate_calculator(var1=spectral_bandwidth_2_s[i], var2=spectral_bandwidth_2_s[j], measure=dtw_kt, multi_agg_func=numpy.mean)
        corr_spectral_bandwidth_2[i, j] = spectral_bandwidth_2_z
corr_spectral_bandwidth_2

In [ ]:
corr_spectral_bandwidth_3 = numpy.full(shape=(len(x_s), len(x_s)), fill_value=numpy.nan)
for i in range(len(x_s)):
    print('{0} / {1}'.format(i, len(x_s)))
    for j in range(len(x_s)):
        spectral_bandwidth_3_z = multivariate_calculator(var1=spectral_bandwidth_3_s[i], var2=spectral_bandwidth_3_s[j], measure=dtw_kt, multi_agg_func=numpy.mean)
        corr_spectral_bandwidth_3[i, j] = spectral_bandwidth_3_z
corr_spectral_bandwidth_3

In [ ]:
corr_spectral_bandwidth_4 = numpy.full(shape=(len(x_s), len(x_s)), fill_value=numpy.nan)
for i in range(len(x_s)):
    print('{0} / {1}'.format(i, len(x_s)))
    for j in range(len(x_s)):
        spectral_bandwidth_4_z = multivariate_calculator(var1=spectral_bandwidth_4_s[i], var2=spectral_bandwidth_4_s[j], measure=dtw_kt, multi_agg_func=numpy.mean)
        corr_spectral_bandwidth_4[i, j] = spectral_bandwidth_4_z
corr_spectral_bandwidth_4

In [ ]:
corr_zero_crossings = numpy.full(shape=(len(x_s), len(x_s)), fill_value=numpy.nan)
for i in range(len(x_s)):
    print('{0} / {1}'.format(i, len(x_s)))
    for j in range(len(x_s)):
        zero_crossings_z = abs(abs(sum(zero_crossings_s[i]) / zero_crossings_s[i].shape[0]) - abs(sum(zero_crossings_s[j]) / zero_crossings_s[j].shape[0]))
        corr_zero_crossings[i, j] = zero_crossings_z
corr_zero_crossings

In [ ]:
corr_mfccs = numpy.full(shape=(len(x_s), len(x_s)), fill_value=numpy.nan)
for i in range(len(x_s)):
    print('{0} / {1}'.format(i, len(x_s)))
    for j in range(len(x_s)):
        mfccs_z = multivariate_calculator(var1=mfccs_s[i].T, var2=mfccs_s[j].T, measure=dtw_kt, multi_agg_func=numpy.mean)
        corr_mfccs[i, j] = mfccs_z
corr_mfccs

In [ ]:
corr_mfccs10 = numpy.full(shape=(len(x_s), len(x_s)), fill_value=numpy.nan)
for i in range(len(x_s)):
    print('{0} / {1}'.format(i, len(x_s)))
    for j in range(len(x_s)):
        mfccs10_z = multivariate_calculator(var1=mfccs10_s[i].T, var2=mfccs10_s[j].T, measure=dtw_kt, multi_agg_func=numpy.mean)
        corr_mfccs10[i, j] = mfccs10_z
corr_mfccs10

In [10]:
corr_chromagram = numpy.full(shape=(len(x_s), len(x_s)), fill_value=numpy.nan)
for i in range(len(x_s)):
    print('{0} / {1}'.format(i, len(x_s)))
    for j in range(len(x_s)):
        chromagram_z = multivariate_calculator(var1=chromagram_s[i].T, var2=chromagram_s[j].T, measure=dtw_kt, multi_agg_func=numpy.mean)
        corr_chromagram[i, j] = chromagram_z
corr_chromagram

0 / 4
1 / 4
2 / 4
3 / 4


array([[1.        , 0.76476007, 0.69818587, 0.73240965],
       [0.76476007, 1.        , 0.69503899, 0.74804317],
       [0.69818587, 0.69503899, 1.        , 0.72264561],
       [0.73240965, 0.74804317, 0.72264561, 1.        ]])

In [ ]:
#####

In [6]:
data = pandas.DataFrame(data={'labels': listed})
data

,labels
0,C:\Users\Edward\Desktop\music\data_sub_mp3\0po...
1,C:\Users\Edward\Desktop\music\data_sub_mp3\9gM...
2,C:\Users\Edward\Desktop\music\data_sub_mp3\sKq...
3,C:\Users\Edward\Desktop\music\data_sub_mp3\xLe...


In [ ]:
clusterer = hdbscan.HDBSCAN(metric='precomputed')
clusterer.fit(corr_spectral_centroids)
data['hdbscan_spectral_centroids'] = clusterer.labels_
data

In [ ]:
clusterer = hdbscan.HDBSCAN(metric='precomputed')
clusterer.fit(corr_spectral_rolloff)
data['hdbscan_spectral_rolloff'] = clusterer.labels_
data

In [ ]:
clusterer = hdbscan.HDBSCAN(metric='precomputed')
clusterer.fit(corr_spectral_bandwidth_2)
data['hdbscan_spectral_bandwidth_2'] = clusterer.labels_
data

In [ ]:
clusterer = hdbscan.HDBSCAN(metric='precomputed')
clusterer.fit(corr_spectral_bandwidth_3)
data['hdbscan_spectral_bandwidth_3'] = clusterer.labels_
data

In [ ]:
clusterer = hdbscan.HDBSCAN(metric='precomputed')
clusterer.fit(corr_spectral_bandwidth_4)
data['hdbscan_spectral_bandwidth_4'] = clusterer.labels_
data

In [ ]:
clusterer = hdbscan.HDBSCAN(metric='precomputed')
clusterer.fit(corr_zero_crossings)
data['hdbscan_zero_crossings'] = clusterer.labels_
data

In [ ]:
clusterer = hdbscan.HDBSCAN(metric='precomputed')
clusterer.fit(corr_mfccs)
data['hdbscan_mfccs'] = clusterer.labels_
data

In [ ]:
clusterer = hdbscan.HDBSCAN(metric='precomputed')
clusterer.fit(corr_mfccs10)
data['hdbscan_mfccs10'] = clusterer.labels_
data

In [13]:
clusterer = hdbscan.HDBSCAN(metric='precomputed')
clusterer.fit(corr_chromagram)
data['hdbscan_chromogram'] = clusterer.labels_
data

HDBSCAN(metric='precomputed')

array([-1, -1, -1, -1], dtype=int64)

In [20]:
clusterer = AgglomerativeClustering(metric='precomputed', linkage='complete')
data['agglo_spectral_centroids'] = clusterer.fit_predict(corr_spectral_centroids)
data

AgglomerativeClustering(linkage='complete', metric='precomputed')

In [22]:
clusterer = AgglomerativeClustering(metric='precomputed', linkage='complete')
data['agglo_spectral_rolloff'] = clusterer.fit_predict(corr_spectral_rolloff)
data

array([0, 1, 1, 0], dtype=int64)

In [23]:
clusterer = AgglomerativeClustering(metric='precomputed', linkage='complete')
data['agglo_spectral_bandwidth_2'] = clusterer.fit_predict(corr_spectral_bandwidth_2)
data

['C:\\Users\\Edward\\Desktop\\music\\data_sub_mp3\\0poLsDf1TvM.mp3',
 'C:\\Users\\Edward\\Desktop\\music\\data_sub_mp3\\9gM7FUXeIb0.mp3',
 'C:\\Users\\Edward\\Desktop\\music\\data_sub_mp3\\sKqfeWQOOZk.mp3',
 'C:\\Users\\Edward\\Desktop\\music\\data_sub_mp3\\xLeGykSUFdY.mp3']

In [234]:
clusterer = AgglomerativeClustering(metric='precomputed', linkage='complete')
data['agglo_spectral_bandwidth_3'] = clusterer.fit_predict(corr_spectral_bandwidth_3)
data

(12, 18813)

In [ ]:
clusterer = AgglomerativeClustering(metric='precomputed', linkage='complete')
data['agglo_spectral_bandwidth_4'] = clusterer.fit_predict(corr_spectral_bandwidth_4)
data

In [ ]:
clusterer = AgglomerativeClustering(metric='precomputed', linkage='complete')
data['agglo_zero_crossings'] = clusterer.fit_predict(corr_zero_crossings)
data

In [198]:
clusterer = AgglomerativeClustering(metric='precomputed', linkage='complete')
data['agglo_mfccs'] = clusterer.fit_predict(corr_mfccs)
data

(3638250,)

In [ ]:
clusterer = AgglomerativeClustering(metric='precomputed', linkage='complete')
data['agglo_mfccs10'] = clusterer.fit_predict(corr_mfccs10)
data

In [151]:
clusterer = AgglomerativeClustering(metric='precomputed', linkage='complete')
data['agglo_chromagram'] = clusterer.fit_predict(corr_chromagram)
data

2

In [73]:
data.to_excel('labeled.xlsx')